# Thermochemistry with PySCF

In this notebook you will use the quantum chemistry package **PySCF** to connect
electronic-structure calculations with macroscopic thermodynamic quantities.
Starting from a molecular geometry, you will

- optimize the structure,
- compute vibrational frequencies, and
- obtain thermodynamic functions such as enthalpy, entropy, and Gibbs free
  energy at a chosen temperature and pressure.

You have already seen in statistical thermodynamics how partition functions
lead to expressions for $U$, $H$, $S$, and $G$. Here you will see how
those ideas are implemented in a modern electronic-structure code.

The notebook begins with a worked example (HF) and then guides you toward a
mini-project: a thermodynamic analysis of the **water–gas shift reaction**

$$
\mathrm{CO + H_2O \rightleftharpoons CO_2 + H_2}
$$

both in the gas phase and in solution.

## Introduction

Thermochemistry links molecular properties to bulk observables such as
reaction energies and equilibrium constants. From statistical mechanics you
know that these quantities can be obtained from a molecular partition
function built from translational, rotational, vibrational, and electronic
energy levels.

In practice, quantum chemistry packages approximate these contributions using
models such as

- **ideal gas translation**,
- **rigid rotor** for rotations, and
- **harmonic oscillator** for vibrations,

combined with an electronic energy from a chosen quantum-chemical method.

This notebook uses PySCF's `hessian.thermo` utilities to assemble these
contributions and report thermodynamic quantities. As you work through the
example, keep the following connections in mind:

- How the normal modes and frequencies from a Hessian calculation enter the
  vibrational partition function.
- How the resulting thermodynamic functions relate to the familiar
  relationships

  $$
  \Delta_r G^\circ(T) = -RT \ln K^\circ(T),
  \quad
  G = H - TS.
  $$

In the final section you will apply the same ideas to the water–gas shift
reaction and compare your calculated equilibrium constants to experimental
data.

In [ ]:
# Import the main PySCF modules used in this workflow.
from pyscf import dft, gto
from pyscf.geomopt.geometric_solver import optimize
from pyscf.hessian import thermo

In [ ]:
# Fix for rotational constants of (near-)linear molecules.
# -------------------------------------------------------
# PySCF's default implementation of `rotation_const` can return very small
# negative values for rotational constants due to numerical noise,
# especially for linear or nearly linear molecules. This helper replaces the
# original routine by a version that:
#   - calls the original implementation,
#   - takes the absolute value of each constant, and
#   - sorts the constants in ascending order.
# This makes the subsequent thermochemistry analysis more robust.

import numpy as np

# Store the original implementation once so we can always call it inside
# our wrapper.
if not hasattr(thermo, "_orig_rotation_const"):
    thermo._orig_rotation_const = thermo.rotation_const


def rotation_const_abs(mass, coords, unit="GHz"):
    """Return non-negative, sorted rotational constants.

    Parameters
    ----------
    mass : array
        Atomic masses.
    coords : array
        Cartesian coordinates of the atoms.
    unit : str, optional
        Unit for the rotational constants (default: GHz).
    """
    # Call the stored original implementation
    rot = thermo._orig_rotation_const(mass, coords, unit=unit)
    # Fix numerical noise: enforce non-negative rotational constants
    rot = np.abs(rot)
    # Ensure ascending order (helps with interpretation and debugging)
    rot = np.sort(rot)
    return rot


# Tell PySCF to use the more robust version from now on.
thermo.rotation_const = rotation_const_abs

## PySCF Workflow

The calculation in this notebook follows a typical **ab initio thermochemistry
workflow**:

1. **Build the molecule** using Cartesian coordinates and select a basis set.
2. **Run an electronic-structure calculation** (here: DFT with B3LYP-D4).
3. **Optimize the geometry** to locate an energy minimum on the potential
   energy surface.
4. **Compute the Hessian** (matrix of second derivatives) at the optimized
   geometry.
5. **Perform a frequency analysis** to obtain normal modes and vibrational
   frequencies.
6. **Evaluate thermodynamic functions** at a given temperature and pressure
   using the rigid-rotor / harmonic-oscillator / ideal-gas models.

As you read and execute each code cell, identify which step of this workflow
it implements and which quantities are being approximated.

### Molecule Setup

In this section you define the **molecular system** for the calculation.
We use hydrogen fluoride (HF) as a simple diatomic example.

Key ingredients:

- The `atom` block specifies element symbols and Cartesian coordinates
  (in Ångström by default).
- The `basis` keyword chooses the one-electron basis set (here: `def2-TZVP`,
  a triple-zeta basis with polarization).
- The `verbose` flag controls how much output PySCF prints.

After running the cell, inspect the printed description of the molecule.

Things to try (after you have run the full workflow once):

- Change the H–F distance and see how it affects the optimized geometry and
  thermochemistry.
- Replace HF by another small, closed-shell molecule (e.g. CO, HCl, H₂O) and
  see which steps of the workflow still work without modification.

In [ ]:
# Define the molecular system.
# ----------------------------
# `gto.M` builds a molecule object from a list of atoms and a chosen basis set.
# Here we specify HF with a simple linear geometry and a reasonably large
# triple-zeta basis (`def2-TZVP`).

mol = gto.M(
    atom="""
    H 0.0 0.0 0.0
    F 0.0 0.0 1.0
    """,  # Cartesian coordinates in Å
    basis="def2-TZVP",  # Orbital basis set
    verbose=3,  # Print a moderate amount of output
)

# Exercise: try changing the H–F distance (the last coordinate) and re-run
# the full workflow. How does the optimized bond length and the computed
# thermochemistry change?

### Geometry Optimization

Once the molecule is defined, the next step is to **optimize the geometry**.

Conceptually:

- We are searching the potential energy surface for a local minimum.
- At a minimum, the gradient (forces on all atoms) vanishes.
- The optimized structure is then used as the reference point for the Hessian
  and vibrational analysis.

In this cell you will

- set up a DFT calculation with the B3LYP-D4 functional,
- optionally embed the molecule in a polarizable continuum model (PCM) to
  mimic solvent effects (here: water), and
- call the geometry optimizer to relax the structure.

After you run the cell, look at the output:

- How many optimization steps were needed?
- Has the H–F distance changed relative to the initial guess?
- What is the final electronic energy at the optimized geometry?

In [ ]:
# Set up the electronic-structure method and optimize the geometry.
# -----------------------------------------------------------------
# We use a restricted Kohn–Sham (RKS) DFT calculation with the B3LYP-D4
# functional. The "D4" term adds an empirical dispersion correction, which
# improves noncovalent interactions compared to plain B3LYP.

# Gas-phase DFT reference.
mf = dft.RKS(mol)
mf.xc = "B3LYP-D4"  # Exchange–correlation functional

# Optionally, add a polarizable continuum model (PCM) to mimic solvent.
# Here we use water as an example (dielectric constant eps ≈ 78.4).
mf = mf.PCM()
mf.with_solvent.eps = 78.3553  # Static dielectric constant of water

# Optimize the geometry at this level of theory.
# `optimize` will repeatedly compute energies and gradients until the
# forces on all atoms fall below a convergence threshold.
mol_opt = optimize(mf)

# After running this cell, inspect the printed output to see how the
# geometry changes during optimization.

### Thermochemistry

The final stage of the workflow is to compute **thermodynamic functions** from
the vibrational frequencies and rotational constants.

In the last code cell of the HF example you will

- run a single-point energy calculation at the optimized geometry,
- compute the Hessian and perform a harmonic frequency analysis,
- evaluate thermodynamic properties at a specified temperature `T` and
  pressure `P` using `thermo.thermo`, and
- print a summary table with quantities such as internal energy, enthalpy,
  entropy, heat capacity, and Gibbs free energy.

After executing that cell, take a moment to interpret the output:

- Identify the contributions from translation, rotation, and vibration.
- Compare the electronic energy to the enthalpy and Gibbs free energy.
- Try changing `T` (for example, 200 K or 400 K) and see how the entropy and
  Gibbs free energy respond.

In the project section you will repeat these steps for several molecules and
combine the results to obtain reaction thermodynamics for the water–gas shift
reaction.

In [ ]:
# Single-point energy, vibrational analysis, and thermochemistry.
# ---------------------------------------------------------------
# At this stage we:
#   1. Rebuild the DFT model on the optimized geometry.
#   2. Compute a single-point electronic energy.
#   3. Evaluate the Hessian (second derivatives) at that point.
#   4. Convert the Hessian to normal modes and harmonic frequencies.
#   5. Use these frequencies to obtain thermodynamic quantities at
#      a specified temperature and pressure.

# Run single point calculation on the optimized geometry (same level & solvent)
mf_opt = dft.RKS(mol_opt)
mf_opt.xc = "B3LYP-D4"
mf_opt = mf_opt.PCM()
mf_opt.with_solvent.eps = 78.3553
energy = mf_opt.kernel()  # Electronic energy at the optimized structure

# Compute the Hessian (matrix of second derivatives of the energy).
hess_opt = mf_opt.Hessian().kernel()

# Frequency analysis (normal modes, frequencies, etc.).
freq_info = thermo.harmonic_analysis(mol_opt, hess_opt)

# Thermochemistry: choose temperature (K) and pressure (Pa).
T = 298.15  # Kelvin (room temperature)
P = 101325.0  # Pascals (1 atm)
thermo_info = thermo.thermo(mf_opt, freq_info["freq_au"], T, P)

# Pretty-print thermochemical data.
# The output typically includes:
#   - Zero-point vibrational energy (ZPVE)
#   - Thermal corrections to energy and enthalpy
#   - Entropy contributions (translational, rotational, vibrational)
#   - Heat capacities
#   - Gibbs free energy at the chosen T and P
thermo.dump_thermo(mf_opt.mol, thermo_info)

# Exercises:
# - Change T to a different temperature (e.g., 200 K or 400 K) and observe
#   how the entropy and Gibbs free energy change.
# - Try running the workflow for a different small molecule and compare the
#   relative magnitudes of the vibrational contributions.

## Project: Water–Gas Shift Reaction

In the final part of this notebook you will apply the workflow above to the
**water–gas shift reaction**

$$
\mathrm{CO(g) + H_2O(g) \rightleftharpoons CO_2(g) + H_2(g)}.
$$

Your goals are:

1. Compute standard reaction thermodynamics in the **gas phase** at
   $T = 298.15\,\text{K}$ and $p = 1\,\text{atm}$.
2. Repeat the analysis with a **continuum solvation model** (PCM water) to
   mimic aqueous solution.
3. Compare your calculated equilibrium constants to **experimental** data and
   comment on possible sources of deviation (electronic structure method,
   harmonic approximation, solvation model, standard-state conventions, etc.).

The recommended strategy is:

1. For each molecular species (CO, H₂O, CO₂, H₂), perform the full workflow:
   geometry optimization, Hessian, frequency analysis, and thermochemistry.
2. Extract the relevant thermodynamic quantity for each species
   (e.g. Gibbs free energy at the chosen $T, p$).
3. Form the reaction quantity, e.g.

   $$
   \Delta_r G^\circ(T) =
   G^\circ_{\mathrm{CO_2}} + G^\circ_{\mathrm{H_2}}
   - G^\circ_{\mathrm{CO}} - G^\circ_{\mathrm{H_2O}},
   $$

   and then

   $$
   K^\circ(T) = \exp\bigl(-\Delta_r G^\circ(T)/(RT)\bigr).
   $$

4. Compare the calculated $K^\circ(T)$ to values obtained from a data
   table (e.g. NIST or a physical chemistry handbook).

### Step 1 – Build Molecules for the Water–Gas Shift Reaction

First, define molecular geometries for each species:

- CO (linear),
- H₂O (bent),
- CO₂ (linear),
- H₂ (linear).

For this exercise you may use reasonable approximate geometries, e.g. taken
from a textbook or an online database. Consistency is more important than
ultimate accuracy: use the **same level of theory and basis set** for all
species.

Below is a template showing how you might build CO. Extend it to H₂O, CO₂,
and H₂, either by copying cells or by writing a small helper function.

In [ ]:
# Example: build a CO molecule.
# --------------------------------
# Use approximate experimental bond lengths as a starting point. PySCF will
# refine the geometry during optimization.

co = gto.M(
    atom="""
    C 0.0000 0.0000 0.0000
    O 0.0000 0.0000 1.13
    """,  # CO bond length ≈ 1.13 Å
    basis="def2-TZVP",
    verbose=3,
)

# TODO:
# - Define analogous `h2o`, `co2`, and `h2` molecule objects.
# - Use reasonable initial geometries (bond lengths and angles).

### Step 2 – A Helper Function for Thermochemistry

To avoid repeating code, it is convenient to write a small helper function
that

1. takes a molecule object,
2. runs a geometry optimization,
3. performs a Hessian and frequency analysis, and
4. returns the thermochemistry dictionary.

The skeleton below illustrates this idea. You may adapt it to your own
coding style. Note that the exact keys in `thermo_info` (names of the
entries for energy, enthalpy, Gibbs free energy, etc.) can be inspected with
`thermo_info.keys()` in the HF example above.

In [ ]:
def compute_thermo_for_molecule(mol, T=298.15, P=101325.0, use_pcm=False):
    """Run geometry optimization and thermochemistry for a given molecule.

    Parameters
    ----------
    mol : pyscf.gto.Mole
        Molecule object.
    T : float
        Temperature in K.
    P : float
        Pressure in Pa.
    use_pcm : bool
        If True, include PCM solvent (water) in the calculation.

    Returns
    -------
    thermo_info : dict
        Dictionary with thermodynamic quantities as returned by
        `thermo.thermo`.
    """
    # Set up the electronic-structure method
    mf = dft.RKS(mol)
    mf.xc = "B3LYP-D4"

    if use_pcm:
        mf = mf.PCM()
        mf.with_solvent.eps = 78.3553  # water

    # Geometry optimization
    mol_opt = optimize(mf)

    # Rebuild the mean-field object on the optimized geometry
    mf_opt = dft.RKS(mol_opt)
    mf_opt.xc = "B3LYP-D4"
    if use_pcm:
        mf_opt = mf_opt.PCM()
        mf_opt.with_solvent.eps = 78.3553

    # Single-point energy
    energy = mf_opt.kernel()

    # Hessian and frequency analysis
    hess_opt = mf_opt.Hessian().kernel()
    freq_info = thermo.harmonic_analysis(mol_opt, hess_opt)

    # Thermochemistry
    thermo_info = thermo.thermo(mf_opt, freq_info["freq_au"], T, P)

    return thermo_info

### Step 3 – Reaction Gibbs Free Energy and Equilibrium Constant

Now combine the molecular thermochemistry into reaction quantities.

1. Compute thermochemistry for each species in the **gas phase**
   (`use_pcm=False`).
2. Extract the Gibbs free energy for each species from the corresponding
   `thermo_info` dictionary.
3. Form the standard reaction Gibbs free energy
   $\Delta_r G^\circ(T)$ for

   $$
   \mathrm{CO + H_2O \rightleftharpoons CO_2 + H_2}.
   $$

4. Compute the equilibrium constant

   $$
   K^\circ(T) = \exp\bigl(-\Delta_r G^\circ(T)/(RT)\bigr)
   $$

   and compare to an experimental value at the same temperature.

Remember:

- All four species must be treated at the **same level of theory** and with
  the **same basis set**.
- Be clear about the **standard state**: here the model uses ideal-gas
  translational motion at 1 atm by default.

Use the template below as a starting point and fill in the missing pieces
once you have inspected the structure of `thermo_info`.

In [ ]:
import math

T = 298.15  # K
P = 101325.0  # Pa
R = 8.314462618  # J mol^-1 K^-1

# Gas-phase thermochemistry for each species
thermo_co_gas = compute_thermo_for_molecule(co, T=T, P=P, use_pcm=False)
# TODO: thermo_h2o_gas = ...
# TODO: thermo_co2_gas = ...
# TODO: thermo_h2_gas  = ...

# Inspect one dictionary to see available keys:
print("Available keys in thermochemistry output:", thermo_co_gas.keys())

# TODO:
# - Replace 'G_key' by the actual key corresponding to the Gibbs free energy.
G_key = "<insert_key_here>"

# Extract Gibbs free energies (be careful with units: PySCF typically uses Hartree)
G_co_gas = thermo_co_gas[G_key]
# TODO: G_h2o_gas = ...
# TODO: G_co2_gas = ...
# TODO: G_h2_gas  = ...

# Reaction Gibbs free energy (products − reactants)
delta_G_gas = (G_co2_gas + G_h2_gas) - (G_co_gas + G_h2o_gas)

# If the energies are in Hartree, convert to J/mol before using in exp(-ΔG/RT).
# 1 Hartree ≈ 2625.499748 J/mol
hartree_to_jmol = 2625.499748
delta_G_gas_Jmol = delta_G_gas * hartree_to_jmol

K_gas = math.exp(-delta_G_gas_Jmol / (R * T))
print(f"Gas-phase Δ_r G°(T={T} K) = {delta_G_gas_Jmol:.2f} J/mol")
print(f"Gas-phase K°(T={T} K)      = {K_gas:.3e}")

### Step 4 – Including Solvent Effects (PCM)

Repeat the analysis with the PCM water model:

1. Recompute thermochemistry for each species with `use_pcm=True`.
2. Form the reaction Gibbs free energy and equilibrium constant as before.
3. Compare the gas-phase and solution-phase values.

Conceptually, the PCM model adds an approximate solvation free energy for
each species. The reaction free energy then includes differences in solvation
between reactants and products.

Points for discussion in your report:

- Does the PCM model shift the equilibrium toward reactants or products?
- How large is the solvent effect relative to the gas-phase reaction free
  energy?
- Which approximations in the electronic structure model and in the
  thermochemistry treatment are likely to be most important for this
  reaction?

In [ ]:
# Repeat the thermochemistry with PCM water (solution phase).
thermo_co_pcm = compute_thermo_for_molecule(co, T=T, P=P, use_pcm=True)
# TODO: thermo_h2o_pcm = ...
# TODO: thermo_co2_pcm = ...
# TODO: thermo_h2_pcm  = ...

# Extract Gibbs free energies as before
G_co_pcm = thermo_co_pcm[G_key]
# TODO: G_h2o_pcm = ...
# TODO: G_co2_pcm = ...
# TODO: G_h2_pcm  = ...

delta_G_pcm = (G_co2_pcm + G_h2_pcm) - (G_co_pcm + G_h2o_pcm)
delta_G_pcm_Jmol = delta_G_pcm * hartree_to_jmol
K_pcm = math.exp(-delta_G_pcm_Jmol / (R * T))

print(f"PCM Δ_r G°(T={T} K) = {delta_G_pcm_Jmol:.2f} J/mol")
print(f"PCM K°(T={T} K)      = {K_pcm:.3e}")